In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import tensorflow_text
import spacy
import numpy

In [2]:
header_list=["sentiment","text"]
df = pd.read_csv("train_fix.csv",names=header_list)
df=df.sample(n=10000)
#df=df.drop(["news_title","url","reddit_title"],axis=1)


In [3]:
print(df)

         sentiment                                               text
777110           0  This gate is not functional with baseboards th...
27862            1  This flag was given to me as a gift from my wi...
1042995          1  This came in a brand new box perfect for my si...
906474           1  I purchased this crock pot as a Christmas gift...
983466           1  This one is probably easier for some to digest...
...            ...                                                ...
908995           0  I've been satisfied with the earbuds that came...
319486           1  I don't know what album this tracklist is from...
1026365          0  This is by far and away the worst DVD I have e...
1033105          0  I got this book for Christmas last year and I ...
391973           1  This was one of my favorite movies of the 90s....

[10000 rows x 2 columns]


In [4]:
texts=df["text"]
nlp = spacy.load("en_core_web_sm")

In [5]:
tokenized_texts=[]
for text in texts:
    doc=nlp(text)
    filtered_tokens = [token for token in doc if not token.is_stop if not token.is_punct]
    tokenized_texts.append(filtered_tokens)

In [6]:
max=0
for text in tokenized_texts:
    if len(text)>max:
        max=len(text)
print(max)

max=256
expanded_tokenized_texts=[]
for text in tokenized_texts:
    text=text[:max] + [nlp("null")]*(max - len(text))
    expanded_tokenized_texts.append(text)
    

119


In [7]:
normalized_texts=[]
for text in  expanded_tokenized_texts:
    vectors=[]
    for word in text:
        vectors.append((numpy.array(word.vector)))
    normalized_texts.append(numpy.array(vectors))

In [8]:
print(numpy.array(normalized_texts).shape)



(10000, 256, 96)


In [9]:
from sklearn.preprocessing import OneHotEncoder

type_one_hot = OneHotEncoder(sparse=False).fit_transform(
  df.sentiment.to_numpy().reshape(-1, 1)
)

In [10]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
train_news, test_news, y_train, y_test =\
  train_test_split(
    normalized_texts, 
    type_one_hot, 
    test_size=.1, 
    random_state=RANDOM_SEED
  )

In [11]:
train_news=np.array(train_news)
test_news=np.array(test_news)
y_train=np.array(y_train)
y_test=np.array(y_test)


In [12]:
print(train_news[1].shape)
print(test_news.shape)
print(y_train.shape)
print(y_test.shape)

(256, 96)
(1000, 256, 96)
(9000, 2)
(1000, 2)


In [13]:
print(train_news.shape)

(9000, 256, 96)


In [14]:
model = keras.Sequential()

model.add(keras.layers.cell(units, return_sequences=False))

model.add(
  keras.layers.Dense(
    units=96,
    input_shape=(train_news[1].shape[0],train_news[1].shape[1]),
    activation='relu'
  )
)
model.add(
  keras.layers.Dropout(rate=0.5)
)

model.add(keras.layers.GlobalMaxPool1D())

model.add(
  keras.layers.Dense(
    units=32,
    activation='relu'
  )
)
model.add(
  keras.layers.Dropout(rate=0.5)
)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(2, activation='softmax'))
model.compile(
    loss='categorical_crossentropy', 
    optimizer=keras.optimizers.Adam(0.001),
    metrics=['accuracy']
)

2021-11-04 21:55:31.916604: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-04 21:55:36.757501: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-04 21:55:36.759062: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-04 21:55:36.809450: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

AttributeError: module 'keras.api._v2.keras.layers' has no attribute 'cell'

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    train_news, y_train, 
    epochs=30, 
    batch_size=32, 
    validation_split=0.1,
    verbose=1, 
    shuffle=True
)

In [ ]:
plt.plot(history.history['loss'], label='train loss'),
plt.plot(history.history['val_loss'], label='val loss'),
plt.xlabel("epoch")
plt.ylabel("Cross-entropy loss"),

In [ ]:
    plt.plot(history.history['accuracy'], label='train accuracy'),
    plt.plot(history.history['val_accuracy'], label='val accuracy'),
    plt.xlabel("epoch"),
    plt.ylabel("accuracy"),
    plt.legend();

In [ ]:
model.evaluate(test_news, y_test)

In [ ]:
model.save('./Model/')